## Cайбель Тимофей Александрович, 423 группа

In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [100]:
data_train = pd.read_csv('./train.csv', delimiter=';')
data_test = pd.read_csv('./test.csv', delimiter=';')
print (data_train.shape, data_test.shape)
data_train.head(5)

(14312, 124) (3579, 124)


,id_contract,id_client,SIGN_DATE,IP_flag,TARGET,F1100,F1110,F1150,F1160,F1170,...,WINNERNUMBER_95_EVER,SIGNEDNUMBER_95_EVER,SUM_95_EVER,FLAG_DISQUALIFICATION,COUNT_CHANGE_YEAR,COUNT_CHANGE_EVER,BIRTHDATE,AGE,SEX_NAME,CITIZENSHIP_NAME
0,2746,3907,13MAR2018:00:00:00,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,женский,Российская Федерация
1,9843,8677,12SEP2018:00:00:00,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2723,5327,13MAR2018:00:00:00,0,1,7000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1538,5159,05FEB2018:00:00:00,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,женский,Российская Федерация
4,8617,6821,06AUG2018:00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,...,43.0,38.0,10411870.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
def preprocess_date_feature(data, name):
    data[name] = data[name].str.replace('^\d+$', '', regex=True)
    return data

def parse_date_feature(data, name):
    data[name] = pd.to_datetime(data[name], format='%d%b%Y:%H:%M:%S', errors='coerce')
    return data

def process_date_feature(data, name):
    data[name + "_day"] = data[name].dt.day
    data[name + "_month"] = data[name].dt.month
    data[name + "_year"] = data[name].dt.year
    data[name + '_dayofweek'] = data[name].dt.dayofweek
    data[name + '_dayofyear'] = data[name].dt.dayofyear
    data[name] = data[name].values.astype(np.int64)
    return data

def proces_tender(data, name):
    data[f'ADMITTED{name}'] = data[f'ADMITTED{name}'].fillna(0)
    data[f'WINNER{name}'] = data[f'WINNER{name}'].fillna(0)
    data[f'SIGNED{name}'] = data[f'SIGNED{name}'].fillna(0)

    func_ratio = lambda x: -1 if x[f'ADMITTED{name}'] == 0 else x[f'WINNER{name}'] / x[f'ADMITTED{name}']
    data[f'RATIO_WIN_{name}'] = data[[f'WINNER{name}', f'ADMITTED{name}']].apply(func_ratio, axis=1)
    func_ratio = lambda x: -1 if x[f'WINNER{name}'.format(name)] == 0 else x[f'SIGNED{name}'] / x[f'WINNER{name}']
    data[f'RATIO_WIN_SIG_tender_{name}'] = data[[f'WINNER{name}', f'SIGNED{name}']].apply(func_ratio, axis=1)

def makeX(data):
    # предобработка данных
    data['CITIZENSHIP_NAME'] = data['CITIZENSHIP_NAME'].fillna(-1).map({-1: -1, 'Российская Федерация': 4, 'Таджикистан': 3, 'Казахстан': 2, 'Армения': 1})
    data['SEX_NAME'] = data['SEX_NAME'].fillna(0).map({0: 0, 'мужской': 1, 'женский': -1})
    group_names = ['OKFS_GROUP', 'OKOPF_GROUP', 'OKOGU_GROUP', 'OKATO_REGIONCODE', 'OKATO_FED'] + ['WORKERSRANGE', 'OKVED_CODE', 'OKVED_CODE_1', 'OKVED_CODE_2', 'OKVED_CODE_3']
    date_names = ['SIGN_DATE', 'DATEFIRSTREG', 'TAXREG_REGDATE', 'TAXREGPAY_REGDATE', 'BIRTHDATE']
        
    data[['OKVED_CODE_1', 'OKVED_CODE_2', 'OKVED_CODE_3']] = data['OKVED_CODE'].str.split('.', 3, expand=True)
        
    for name in group_names + ['id_client']:
        data[name] = data[name].fillna(0)
        tmp = data[name].value_counts()
        tmp = tmp + 0.1 * np.random.randn(len(tmp))
        data[name] = data[name].map(tmp)

    for name in date_names:
        data = preprocess_date_feature(data, name)
        data = parse_date_feature(data, name)
    data['TAXREG_REGDATE_DIFF_DATEFIRSTREG'] = (data['TAXREG_REGDATE'] - data['DATEFIRSTREG']).astype("timedelta64[D]")
    data['SIGN_DATE_DIFF_DATEFIRSTREG'] = (data['SIGN_DATE'] - data['DATEFIRSTREG']).astype("timedelta64[D]")
    for name in ['TAXREG_REGDATE_DIFF_DATEFIRSTREG', 'SIGN_DATE_DIFF_DATEFIRSTREG']:
        data = parse_date_feature(data, name)
    for name in date_names + ['TAXREG_REGDATE_DIFF_DATEFIRSTREG', 'SIGN_DATE_DIFF_DATEFIRSTREG']:
        data = process_date_feature(data, name)
        

    for tender in ['NUMBER_233_YEAR','NUMBER_233_EVER', 'NUMBER_95_YEAR','NUMBER_95_EVER']:
        proces_tender(data, tender)

    data.fillna(-1, inplace=True)
    
    return data

In [102]:
data_train = makeX(data_train) # обрабатываем обучение
data_test = makeX(data_test) # обрабатываем тест

In [103]:
y_train = data_train.pop('TARGET').values # целевые значения
data_test = data_test[data_train.columns] # на всякий случай - вдруг, перемешаны столбцы

In [104]:
import lightgbm as lgb

model = lgb.LGBMClassifier(num_leaves=15,
                           learning_rate=0.065,
                           n_estimators=100,
                           colsample_bytree=0.7,
                           subsample=0.7,
                           subsample_freq=1,
                           objective='binary')

In [105]:
model.fit(data_train, y_train)

a = model.predict_proba(data_test)[:, 1] # получаем ответ 

In [106]:
df = pd.DataFrame({'id_contract': data_test.id_contract.values, 'TARGET': a})
df.to_csv('ans.csv', sep=',', index=False)